In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [33]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = self.params,
            metric_file_name = config.metrics_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/nasim-raj-laskar/datascienceproject.mlflow"
        )

        return model_evaluation_config

In [34]:
import os
import mlflow
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow.sklearn
import numpy as np
import joblib
from urllib.parse import urlparse

In [35]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def evaluate_model(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop(columns=[self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.evaluate_model(test_y, predicted_qualities)

            scores={
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [36]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-09-12 15:04:21,955: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-12 15:04:21,959: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-12 15:04:21,983: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-12 15:04:21,985: INFO: common: created directory at: artifacts]
[2025-09-12 15:04:21,987: INFO: common: created directory at: artifacts/model_evaluation]
[2025-09-12 15:04:24,651: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/09/12 15:04:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/09/12 15:04:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.
2025/09/12 15:04:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run silent-quail-407 at: https://dagshub.com/nasim-raj-laskar/datascienceproject.mlflow/#/experiments/0/runs/0caffc1b972a434caa8769cde6aa1115.
2025/09/12 15:04:43 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/nasim-raj-laskar/datascienceproject.mlflow/#/experiments/0.
